In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 2
setup_one_gpu(GPU)

Picking GPU 2


In [2]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from functools import partial

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter
from tf_lab.point_clouds.ae_templates import mlp_architecture_ala_iclr_18, default_train_params

from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id
from tf_lab.point_clouds.convenience import reconstruct_pclouds

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [53]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/'
experiment_tag = 'mlp_with_split_3pc_usampled'
loss = 'emd'

n_pc_points = 2048
random_seed = 42

In [5]:
class_name = raw_input('Give me the class type: ').lower()
syn_id = snc_category_to_synth_id()[class_name]

Give me the class type: chair


In [44]:
def load_multiple_version_of_pcs(version, syn_id):
    if version == 'uniform_all':        
        versions = ['centered', 'centered_2nd_version', 'centered_3rd_version']    

    elif version == 'uniform_one':        
        versions = ['centered']
        
    elif version == 'fps':
        versions = ['fps_sampled_in_u_sphere']
    
    elif version == 'all':
        versions = ['centered', 'centered_2nd_version', 'centered_3rd_version', 'fps_sampled_in_u_sphere']
        
    splits = {'train':None, 'val':None, 'test': None}
        
    for s in splits.keys():
        print 'Loading %s data.' % (s,)
        s_file = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/'+syn_id+ '/85_5_10/', s + '.txt')
        print s_file
        for i, v in enumerate(versions):
            top_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/', v, str(n_pc_points))                
            splitter = Data_Splitter(top_pclouds_path, data_file_ending='.ply', random_seed=random_seed)                        
            pcs_in_split = splitter.load_splits(s_file)                
            pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(pcs_in_split, n_threads=20, loader=snc_loader, verbose=True)        
            if splits[s] is None:
                splits[s] = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)
            else:
                splits[s].merge(PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids))
    return splits

In [45]:
in_data = load_multiple_version_of_pcs('uniform_all' ,syn_id)

Loading test data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/test.txt
679 pclouds were loaded. They belong in 1 shape-classes.
679 pclouds were loaded. They belong in 1 shape-classes.
679 pclouds were loaded. They belong in 1 shape-classes.
Loading train data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/train.txt
5761 pclouds were loaded. They belong in 1 shape-classes.
5761 pclouds were loaded. They belong in 1 shape-classes.
5761 pclouds were loaded. They belong in 1 shape-classes.
Loading val data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/val.txt
338 pclouds were loaded. They belong in 1 shape-classes.
338 pclouds were loaded. They belong in 1 shape-classes.
338 pclouds were loaded. They belong in 1 shape-classes.


In [34]:
def retrieve_best_validation_epoch(train_stats_file):
    held_out_mark = 'On Held_Out:'
    held_out_loss = []
    held_out_epoch = []
    with open(train_stats_file, 'r') as f_in:    
        for line in f_in:
            if line.startswith(held_out_mark):
                line_datum = line.replace(held_out_mark, '')
                tokens = line_datum.split()        
                held_out_loss.append(float(tokens[1]))
                held_out_epoch.append(int(tokens[0]))
    held_out_loss = np.array(held_out_loss)
    held_out_epoch = np.array(held_out_epoch)
    best_idx = np.argmin(held_out_loss)
    return held_out_loss[best_idx], held_out_epoch[best_idx]

In [54]:
bneck_list = [4, 8, 16, 32, 64, 128, 256]

for bneck in bneck_list:
    experiment_id = '_'.join(['ae', class_name, experiment_tag, str(n_pc_points), 'pts', str(bneck), 'bneck', loss])
    train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/', experiment_id)
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    val_error, best_epoch = retrieve_best_validation_epoch(osp.join(train_dir, 'train_stats.txt'))
    print bneck, val_error, best_epoch
    
    if best_epoch % conf.saver_step != 0: # Model was not saved at that epoch.
        best_epoch += best_epoch % conf.saver_step
    
    print best_epoch
    
    conf.encoder_args['verbose'] = False
    conf.decoder_args['verbose'] = False
    
    reset_tf_graph()
    ae = PointNetAutoEncoder(conf.experiment_name, conf)    
    ae.restore_model(conf.train_dir, best_epoch, verbose=True)

    _, l_val = reconstruct_pclouds(ae, in_data['val'].point_clouds, batch_size=100)    
    _, l_test = reconstruct_pclouds(ae, in_data['test'].point_clouds, batch_size=100)
    _, l_train = reconstruct_pclouds(ae, in_data['train'].point_clouds, batch_size=100)
    print l_val, l_test, l_train

4 0.001534496 300
300
Model restored in epoch 300.
0.00160430411806 0.00153939216032 0.00130340292781
